# Neo4j와 LangChain을 활용한 ETF 상품 추천

---

## 1. Neo4J Desktop 환경 설정

- **Neo4j Desktop 소개**:
    - Neo4j 작업을 위한 클라이언트 애플리케이션
    - 로컬 환경에서 Neo4j를 학습하고 실험하는 데 필요한 모든 것을 포함함
    - 사용자의 컴퓨터 리소스가 허용하는 한 **여러 로컬 데이터베이스**를 생성할 수 있음
    - **Enterprise Edition 라이센스**: 단, 개발자 개인에 대해서는 1개 계정을 테스트 목적으로 지원

- **다운로드 및 설치**: https://neo4j.com/deployment-center/?desktop-gdb
    - Neo4J 5.24.0 선택
    - 새 프로젝트 생성 및 DBMS 추가

- **APOC 플러그인 설정**: 
    - APOC 플러그인을 설치하려는 데이터베이스가 있는 프로젝트(Graph DBMS)를 선택
    - Graph DBMS 메뉴 클릭하고, APOC 플러그인(Plugin) 설치

- **설정 파일 수정**: 데이터베이스를 중지한 상태에서 데이터베이스 카드의 오른쪽에 있는 `...` (메뉴) 버튼을 클릭

    - 메뉴에서 **Settings** 선택하고 다음 내용을 추가 (`neo4j.conf` 파일)
        ```
        dbms.security.procedures.unrestricted=apoc.meta.*,apoc.*
        ```

In [ ]:
import os
from dotenv import load_dotenv

# 환경 변수 로드
load_dotenv()

In [ ]:
from langchain_neo4j import Neo4jGraph

# Neo4j Desktop 연결 설정
graph = Neo4jGraph(
    url=os.getenv("NEO4J_URI"),
    username=os.getenv("NEO4J_USERNAME"),
    password=os.getenv("NEO4J_PASSWORD"),
    database=os.getenv("NEO4J_DATABASE"),
    enhanced_schema=True
)

In [ ]:
# 테스트 쿼리 실행 
cypher_query = """
MATCH (n) 
RETURN count(n) AS node_count
"""

graph.query(cypher_query)

In [ ]:
def reset_database(graph):
    """
    APOC 없이 데이터베이스 초기화하기
    """
    # 모든 노드와 관계 삭제
    graph.query("MATCH (n) DETACH DELETE n")
    
    # 모든 제약조건 삭제
    constraints = graph.query("SHOW CONSTRAINTS")
    for constraint in constraints:
        constraint_name = constraint.get("name")
        if constraint_name:
            graph.query(f"DROP CONSTRAINT {constraint_name}")
    
    # 모든 인덱스 삭제
    indexes = graph.query("SHOW INDEXES")
    for index in indexes:
        index_name = index.get("name")
        index_type = index.get("type")
        if index_name and index_type != "CONSTRAINT":
            graph.query(f"DROP INDEX {index_name}")
    
    print("데이터베이스가 초기화되었습니다.")

# 데이터베이스 초기화
reset_database(graph)

---

## 2. **Knowledge Graph 구축**

### 2.1 ETF 데이터 전처리


#### 1) **데이터셋 준비**

| 필드명 | 설명 |
|-------|------|
| `id` | 고유 식별자 |
| `korean_name` | 한글명 |
| `english_name` | 영문명 |
| `code` | 종목코드 |
| `listing_date` | 상장일 |
| `fund_type` | 펀드형태 |
| `index_name` | 기초지수명 |
| `tracking_multiplier` | 추적배수 |
| `management_company` | 자산운용사 |
| `ap_company` | 지정참가회사(AP) |
| `total_fee` | 총보수(%) |
| `tax_type` | 과세유형 |
| `website` | 홈페이지 |
| `base_market` | 기초 시장 |
| `base_asset` | 기초 자산 |
| `basic_info` | 기본 정보 |
| `investment_notice` | 투자유의사항 |

In [ ]:
# ETF 데이터 (CSV에서 로드)
import pandas as pd

# CSV 파일 로드
df = pd.read_csv("etf_data/etf_info_cleaned.csv", encoding="utf-8")

# 데이터 형태 확인
print(f"데이터 형태: {df.shape}")

# 데이터 샘플 확인
df.head(2)

#### 2) **ETF 속성에서 고유명사/카테고리 추출**

- ETF 데이터에서 엔티티 추출하는 함수

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
from typing import List, Dict, Any, Optional, Union
from tqdm import tqdm
import pandas as pd

# ETF에서 고유명사와 카테고리 추출을 위한 Pydantic 모델 정의
class ETFEntities(BaseModel):
    """ETF 상품에서 추출한 고유명사와 카테고리 정보"""
    companies: Optional[List[str]] = Field(default=None, description="ETF에 포함된 회사/기업 이름")
    sectors: Optional[List[str]] = Field(default=None, description="ETF가 속한 산업 섹터")
    technologies: Optional[List[str]] = Field(default=None, description="ETF 관련 기술 키워드")
    markets: Optional[List[str]] = Field(default=None, description="ETF 관련 시장/지역")
    asset_classes: Optional[List[str]] = Field(default=None, description="ETF 자산 클래스(주식, 채권, 원자재 등)")
    investment_themes: Optional[List[str]] = Field(default=None, description="ETF 투자 테마(AI, 친환경, 메타버스 등)")


def extract_entities_from_etf(row: pd.Series) -> ETFEntities:
    """
    ETF 메타데이터에서 고유명사와 카테고리를 추출합니다.
    
    Args:
        row (pd.Series): ETF 데이터 행
    
    Returns:
        ETFEntities: 추출된 고유명사와 카테고리 목록
    """
    # 프롬프트 템플릿 정의
    prompt = ChatPromptTemplate.from_messages([
        ("system", """ETF 상품의 고유명사와 카테고리 추출 전문가입니다. 
        제공된 ETF 상품 정보에서 다음 항목들을 정확하게 리스트로 추출하세요:
        
        추출 가이드라인:
        - companies: ETF에 포함된 또는 관련된 회사/기업 이름. 
        - sectors: ETF가 속한 산업 섹터(IT, 금융, 헬스케어 등)
        - technologies: ETF 관련 기술 키워드(AI, 블록체인, 클라우드 등)
        - markets: ETF 관련 시장/지역(국내, 미국, 아시아 등)
        - asset_classes: ETF 자산 클래스(주식, 채권, 원자재 등)
        - investment_themes: ETF 투자 테마(AI, 친환경, 메타버스 등)
        
        각 항목은 개별 리스트에 담아야 하며, 명확한 증거가 있는 항목만 포함하세요.
        ETF 이름, 설명, 기초 지수 등에서 관련 정보를 파악하세요.
        항목이 없는 경우 빈 리스트를 반환하세요.
        """),
        ("human", """다음 ETF 상품에서 고유명사와 카테고리를 추출하세요:
        
        ETF ID: {id}
        한글명: {korean_name}
        영문명: {english_name}
        종목코드: {code}
        상장일: {listing_date}
        펀드형태: {fund_type}
        기초지수명: {index_name}
        기초 시장: {base_market}
        기초 자산: {base_asset}
        기본 정보: {basic_info}
        """)
    ])

    # LLM 및 체인 설정
    llm = ChatOpenAI(model="gpt-4.1", temperature=0)    
    
    # 구조화된 출력을 위해 Pydantic 모델과 연결
    llm_with_structured_output = llm.with_structured_output(ETFEntities)
    
    # 프롬프트와 LLM을 연결하는 체인 구성
    chain = prompt | llm_with_structured_output
    
    # 엔티티 추출 시도
    try:
        # 데이터 행에서 필드 추출
        base_market_str = row['base_market'] if 'base_market' in row else ""
        base_asset_str = row['base_asset'] if 'base_asset' in row else ""
        basic_info = row['basic_info'] if 'basic_info' in row else ""
        
        # LLM을 통해 엔티티 추출
        entities = chain.invoke({
            "id": row['id'] if 'id' in row else "",
            "korean_name": row['korean_name'] if 'korean_name' in row else "",
            "english_name": row['english_name'] if 'english_name' in row else "",
            "code": row['code'] if 'code' in row else "",
            "listing_date": row['listing_date'] if 'listing_date' in row else "",
            "fund_type": row['fund_type'] if 'fund_type' in row else "",
            "index_name": row['index_name'] if 'index_name' in row else "",
            "base_market": base_market_str,
            "base_asset": base_asset_str,
            "basic_info": basic_info
        })
        
        return entities
    except Exception as e:
        # 에러 발생 시 로그 출력 및 빈 엔티티 반환
        print(f"ETF 엔티티 추출 중 오류 발생: {e}")
        return ETFEntities(
            companies=[],
            sectors=[],
            technologies=[],
            markets=[],
            asset_classes=[],
            investment_themes=[]
        )
    
# 첫 번째 ETF 행에서 엔티티 추출
first_row = df.iloc[0]
entities = extract_entities_from_etf(first_row)

entities

- 모든 ETF에서 엔티티 추출 (gpt-4.1 사용)

In [ ]:
# 모든 ETF에서 엔티티 추출하는 함수
def extract_all_entities(df):
    """
    모든 ETF에서 엔티티를 추출하고 결과를 저장합니다.
    
    Args:
        df (pd.DataFrame): ETF 데이터프레임
        
    Returns:
        dict: ETF ID를 키로, 추출된 엔티티를 값으로 하는 딕셔너리
    """
    all_entities = {}
    
    # 모든 ETF에 대해 엔티티 추출
    for idx, row in tqdm(df.iterrows()):
        etf_id = row['id']
        entities = extract_entities_from_etf(row)
        all_entities[etf_id] = entities
    
    return all_entities

# 모든 ETF에서 엔티티 추출
all_etf_entities = extract_all_entities(df)

In [ ]:
print(f"추출된 엔티티 수: {len(all_etf_entities)}")
print(f"첫 번째 ETF의 엔티티: {all_etf_entities[list(all_etf_entities.keys())[0]]}")

In [ ]:
# 추출한 엔티티를 별도로 저장
import pickle 
with open('etf_data/all_etf_entities.pkl', 'wb') as f:
    pickle.dump(all_etf_entities, f)

- 모든 엔티티에 대한 통계 처리

In [ ]:
from pydantic import BaseModel, Field
from typing import List, Dict, Any, Optional

# ETF에서 고유명사와 카테고리 추출을 위한 Pydantic 모델 정의
class ETFEntities(BaseModel):
    """ETF 상품에서 추출한 고유명사와 카테고리 정보"""
    companies: Optional[List[str]] = Field(default=None, description="ETF에 포함된 회사/기업 이름")
    sectors: Optional[List[str]] = Field(default=None, description="ETF가 속한 산업 섹터")
    technologies: Optional[List[str]] = Field(default=None, description="ETF 관련 기술 키워드")
    markets: Optional[List[str]] = Field(default=None, description="ETF 관련 시장/지역")
    asset_classes: Optional[List[str]] = Field(default=None, description="ETF 자산 클래스(주식, 채권, 원자재 등)")
    investment_themes: Optional[List[str]] = Field(default=None, description="ETF 투자 테마(AI, 친환경, 메타버스 등)")

In [ ]:
# 추출한 엔티티를 로드
import pickle
with open('etf_data/all_etf_entities.pkl', 'rb') as f:
    etf_entities = pickle.load(f)

print(f"로드된 엔티티 수: {len(etf_entities)}")
print(f"첫 번째 ETF의 엔티티: {etf_entities[list(etf_entities.keys())[0]]}")

In [ ]:
# 엔티티 통계 수집 함수
def collect_entity_statistics(all_entities):
    """
    추출된 모든 엔티티에서 통계를 수집합니다.
    
    Args:
        all_entities (dict): ETF ID를 키로, 추출된 엔티티를 값으로 하는 딕셔너리
        
    Returns:
        dict: 엔티티 유형별 통계 정보
    """
    # 엔티티 유형별 통계 초기화
    stats = {
        'companies': set(),
        'sectors': set(),
        'technologies': set(),
        'markets': set(),
        'asset_classes': set(),
        'investment_themes': set()
    }
    
    # 모든 ETF의 엔티티 수집
    for etf_id, entities in all_entities.items():
        for entity_type in stats.keys():
            entity_list = getattr(entities, entity_type, [])
            stats[entity_type].update(entity_list)
    
    # set을 list로 변환
    for key in stats:
        stats[key] = sorted(list(stats[key]))
    
    return stats

# 엔티티 통계 수집
entity_stats = collect_entity_statistics(etf_entities)

# 결과 출력
for entity_type, entities in entity_stats.items():
    print(f"{entity_type}: {len(entities)}개")
    print(f"  {', '.join(entities)}")

In [ ]:
# ETF 데이터프레임에 엔티티 정보 추가

df_with_entities = df.copy()
for etf_id, entities in etf_entities.items():
    # 해당 ETF 행 찾기
    idx = df_with_entities[df_with_entities['id'] == etf_id].index
    if len(idx) > 0:
        # 엔티티 정보 추가 - 배열을 |로 결합하여 문자열로 저장
        df_with_entities.loc[idx[0], 'extracted_companies'] = '|'.join(entities.companies) if entities.companies else ''
        df_with_entities.loc[idx[0], 'extracted_sectors'] = '|'.join(entities.sectors) if entities.sectors else ''
        df_with_entities.loc[idx[0], 'extracted_technologies'] = '|'.join(entities.technologies) if entities.technologies else ''
        df_with_entities.loc[idx[0], 'extracted_markets'] = '|'.join(entities.markets) if entities.markets else ''
        df_with_entities.loc[idx[0], 'extracted_asset_classes'] = '|'.join(entities.asset_classes) if entities.asset_classes else ''
        df_with_entities.loc[idx[0], 'extracted_investment_themes'] = '|'.join(entities.investment_themes) if entities.investment_themes else ''

# 엔티티가 추가된 ETF 데이터 확인
df_with_entities.head()

In [ ]:
# 데이터프레임 저장
df_with_entities.to_csv("etf_data/etf_with_extracted_entities.csv", index=False, encoding='utf-8-sig')

### 2.2 KG 온톨로지 구현

#### 1) **스키마 정의**

- **노드 유형**:
  - ETF: 상장지수펀드 정보를 담는 노드 (id, 이름, 코드, 상장일 등 속성 포함)
  - AssetManager: 자산운용사 정보를 담는 노드 (이름, 관리 ETF 수 등 속성 포함)
  - Sector: 산업 섹터 정보 노드 (예: AI 프로세스칩, 시스템 반도체)
  - Technology: 기술 정보 노드 (예: AI, 자연어처리)
  - Market: 시장 정보 노드 (예: 국내, 해외, 코스피)
  - AssetClass: 자산 유형 노드 (예: 주식, 채권)
  - InvestmentTheme: 투자 테마 노드 (예: AI)

- **관계 유형**:
  - MANAGED_BY: ETF와 자산운용사 간의 관계 (ETF → AssetManager)
  - FOCUSES_ON: ETF와 섹터/기술/테마 간의 관계 (ETF → Sector/Technology/InvestmentTheme)
  - INVESTS_IN: ETF와 시장/자산유형 간의 관계 (ETF → Market/AssetClass)

In [ ]:
# 데이터프레임 로드
import pandas as pd
df_with_entities = pd.read_csv("etf_data/etf_with_extracted_entities.csv", encoding='utf-8-sig')

df_with_entities.head()

#### 2) **제약조건 설정**

In [ ]:
# Neo4j 데이터베이스에 Cypher 쿼리를 사용하여 제약조건 설정
# 제약조건은 노드의 특정 속성이 고유(UNIQUE)하도록 보장하여 데이터 중복을 방지함
constraints = [
    # ETF 노드의 id 속성이 고유하도록 제약조건 설정
    # 이를 통해 동일한 id를 가진 ETF가 중복 저장되는 것을 방지
    "CREATE CONSTRAINT IF NOT EXISTS FOR (e:ETF) REQUIRE e.id IS UNIQUE",
    
    # AssetManager 노드의 id 속성이 고유하도록 제약조건 설정
    # 동일한 id의 자산운용사가 여러 번 생성되는 것을 방지
    "CREATE CONSTRAINT IF NOT EXISTS FOR (c:AssetManager) REQUIRE c.id IS UNIQUE",
    
    # Sector 노드의 id 속성이 고유하도록 제약조건 설정
    # 동일한 id의 섹터가 중복 생성되는 것을 방지
    "CREATE CONSTRAINT IF NOT EXISTS FOR (s:Sector) REQUIRE s.id IS UNIQUE",
    
    # Technology 노드의 id 속성이 고유하도록 제약조건 설정
    # 동일한 id의 기술이 중복 생성되는 것을 방지
    "CREATE CONSTRAINT IF NOT EXISTS FOR (t:Technology) REQUIRE t.id IS UNIQUE",
    
    # Market 노드의 id 속성이 고유하도록 제약조건 설정
    # 동일한 id의 시장이 중복 생성되는 것을 방지
    "CREATE CONSTRAINT IF NOT EXISTS FOR (m:Market) REQUIRE m.id IS UNIQUE",
    
    # AssetClass 노드의 id 속성이 고유하도록 제약조건 설정
    # 동일한 id의 자산 클래스가 중복 생성되는 것을 방지
    "CREATE CONSTRAINT IF NOT EXISTS FOR (a:AssetClass) REQUIRE a.id IS UNIQUE",
    
    # InvestmentTheme 노드의 id 속성이 고유하도록 제약조건 설정
    # 동일한 id의 투자 테마가 중복 생성되는 것을 방지
    "CREATE CONSTRAINT IF NOT EXISTS FOR (i:InvestmentTheme) REQUIRE i.id IS UNIQUE",
]

# 정의된 모든 제약조건을 순회하며 Neo4j 데이터베이스에 적용
# graph.query() 메서드를 사용하여 각 Cypher 쿼리를 실행
for constraint in constraints:
    graph.query(constraint)

In [ ]:
from langchain_neo4j.graphs.graph_document import GraphDocument, Node, Relationship
from langchain_core.documents import Document
from tqdm import tqdm
import pandas as pd
import numpy as np

# 중복 노드 생성을 방지하기 위한 딕셔너리 초기화
node_dict = {}  # 노드 ID를 키로 사용하여 생성된 노드 객체를 저장

# 노드 간 관계를 저장할 리스트 초기화
relationships = []

# ETF 데이터프레임을 순회하며 그래프 구조로 변환
for _, row in tqdm(df_with_entities.iterrows(), total=len(df_with_entities), desc="ETF 온톨로지 구축 중"):
    # ETF ID 생성
    etf_id = row['id']
    
    # ETF 노드 생성 (이미 존재하는지 확인하여 중복 방지)
    if etf_id not in node_dict:
        # ETF 노드 속성 설정
        etf_properties = {
            "id": etf_id,
            "name": row['korean_name'],
            "english_name": row['english_name'],
            "code": row['code'],
            "listing_date": row['listing_date'],
            "fund_type": row['fund_type'],
            "total_fee": row['total_fee'],
            "website": row['website'],
            "basic_info": row['basic_info'],
            "investment_notice": row['investment_notice'],
            "index_name": row['index_name'],
            "tracking_multiplier": row['tracking_multiplier'],
            "ap_company": row['ap_company'],
            "tax_type": row['tax_type'],
            "base_market": row['base_market'],
            "base_asset": row['base_asset'],
        }
        
        # ETF 노드 객체 생성
        etf_node = Node(
            id=etf_id,
            type="ETF",  # 노드 유형 지정
            properties=etf_properties
        )
        
        # 생성된 ETF 노드를 딕셔너리에 저장
        node_dict[etf_id] = etf_node
    
    # 자산운용사 노드 생성 및 연결
    if 'management_company' in row and pd.notna(row['management_company']):
        company_name = row['management_company']
        company_id = f"company-{company_name}"
        
        # 자산운용사 노드가 아직 생성되지 않았다면 새로 생성
        if company_id not in node_dict:
            company_node = Node(
                id=company_id,
                type="AssetManager",
                properties={"name": company_name}
            )
            node_dict[company_id] = company_node
        
        # ETF와 자산운용사 간의 'MANAGED_BY' 관계 생성 (ETF → AssetManager)
        relationships.append(
            Relationship(
                source=node_dict[etf_id],
                target=node_dict[company_id],
                type="MANAGED_BY",
                properties={}
            )
        )
    
    # 섹터 노드 생성 및 연결
    if 'extracted_sectors' in row and pd.notna(row['extracted_sectors']):
        sectors = str(row['extracted_sectors']).split('|')
        for sector in sectors:
            if not sector:
                continue
            sector_id = f"sector-{sector}"
            
            # 섹터 노드가 아직 생성되지 않았다면 새로 생성
            if sector_id not in node_dict:
                sector_node = Node(
                    id=sector_id,
                    type="Sector",
                    properties={"name": sector}
                )
                node_dict[sector_id] = sector_node
            
            # ETF와 섹터 간의 'FOCUSES_ON' 관계 생성 (ETF → Sector)
            relationships.append(
                Relationship(
                    source=node_dict[etf_id],
                    target=node_dict[sector_id],
                    type="FOCUSES_ON",
                    properties={}
                )
            )
    
    # 기술 노드 생성 및 연결
    if 'extracted_technologies' in row and pd.notna(row['extracted_technologies']):
        technologies = str(row['extracted_technologies']).split('|')
        for tech in technologies:
            if not tech:
                continue
            tech_id = f"tech-{tech}"
            
            # 기술 노드가 아직 생성되지 않았다면 새로 생성
            if tech_id not in node_dict:
                tech_node = Node(
                    id=tech_id,
                    type="Technology",
                    properties={"name": tech}
                )
                node_dict[tech_id] = tech_node
            
            # ETF와 기술 간의 'FOCUSES_ON' 관계 생성 (ETF → Technology)
            relationships.append(
                Relationship(
                    source=node_dict[etf_id],
                    target=node_dict[tech_id],
                    type="FOCUSES_ON",
                    properties={}
                )
            )
    
    # 시장 노드 생성 및 연결
    if 'extracted_markets' in row and pd.notna(row['extracted_markets']):
        markets = str(row['extracted_markets']).split('|')
        for market in markets:
            if not market:
                continue
            market_id = f"market-{market}"
            
            # 시장 노드가 아직 생성되지 않았다면 새로 생성
            if market_id not in node_dict:
                market_node = Node(
                    id=market_id,
                    type="Market",
                    properties={"name": market}
                )
                node_dict[market_id] = market_node
            
            # ETF와 시장 간의 'INVESTS_IN' 관계 생성 (ETF → Market)
            relationships.append(
                Relationship(
                    source=node_dict[etf_id],
                    target=node_dict[market_id],
                    type="INVESTS_IN",
                    properties={}
                )
            )
    
    # 자산 유형 노드 생성 및 연결
    if 'extracted_asset_classes' in row and pd.notna(row['extracted_asset_classes']):
        asset_classes = str(row['extracted_asset_classes']).split('|')
        for asset_class in asset_classes:
            if not asset_class:
                continue
            asset_id = f"asset-{asset_class}"
            
            # 자산 유형 노드가 아직 생성되지 않았다면 새로 생성
            if asset_id not in node_dict:
                asset_node = Node(
                    id=asset_id,
                    type="AssetClass",
                    properties={"name": asset_class}
                )
                node_dict[asset_id] = asset_node
            
            # ETF와 자산 유형 간의 'INVESTS_IN' 관계 생성 (ETF → AssetClass)
            relationships.append(
                Relationship(
                    source=node_dict[etf_id],
                    target=node_dict[asset_id],
                    type="INVESTS_IN",
                    properties={}
                )
            )
    
    # 투자 테마 노드 생성 및 연결
    if 'extracted_investment_themes' in row and pd.notna(row['extracted_investment_themes']):
        themes = str(row['extracted_investment_themes']).split('|')
        for theme in themes:
            if not theme:
                continue
            theme_id = f"theme-{theme}"
            
            # 투자 테마 노드가 아직 생성되지 않았다면 새로 생성
            if theme_id not in node_dict:
                theme_node = Node(
                    id=theme_id,
                    type="InvestmentTheme",
                    properties={"name": theme}
                )
                node_dict[theme_id] = theme_node
            
            # ETF와 투자 테마 간의 'FOCUSES_ON' 관계 생성 (ETF → InvestmentTheme)
            relationships.append(
                Relationship(
                    source=node_dict[etf_id],
                    target=node_dict[theme_id],
                    type="FOCUSES_ON",
                    properties={}
                )
            )

# 노드 딕셔너리에서 모든 노드 객체를 리스트 형태로 추출
nodes = list(node_dict.values())

# GraphDocument 객체 생성
graph_doc = GraphDocument(
    nodes=nodes,
    relationships=relationships
)

# 생성된 GraphDocument를 Neo4j 데이터베이스에 저장
graph.add_graph_documents([graph_doc])

print(f"총 노드 수: {len(node_dict)}")
print(f"총 관계 수: {len(relationships)}")
print("ETF 온톨로지 구축 완료!")

In [ ]:
# Neo4j 데이터베이스의 현재 스키마 정보를 새로고침하여 최신 상태로 업데이트
graph.refresh_schema()
print(graph.schema)

In [ ]:
# AI 관련 ETF 검색 예시
cypher_query = """
// ETF 노드와 Technology 노드 간의 FOCUSES_ON 관계를 가진 패턴 매칭
MATCH (e:ETF)-[:FOCUSES_ON]->(t:Technology)
// 기술 이름에 'AI' 또는 '인공지능'이 포함된 노드만 필터링
WHERE t.name CONTAINS 'AI' OR t.name CONTAINS '인공지능'
// ETF의 id, 이름, 영문 이름, 코드 정보 반환
RETURN e.id, e.name, e.english_name, e.code
// 이름을 기준으로 오름차순 정렬
ORDER BY e.name
"""
# Neo4j 데이터베이스에 쿼리 실행
result = graph.query(cypher_query)

# 결과 출력
for row in result:
    print(row)

In [ ]:
# 특정 섹터와 기술을 모두 포함하는 ETF 검색 예시
cypher_query = """
// ETF 노드와 Sector, Technology 노드 간의 관계 매칭
MATCH (e:ETF)-[:FOCUSES_ON]->(s:Sector), (e)-[:FOCUSES_ON]->(t:Technology)
// 반도체 섹터와 AI 기술을 가진 ETF 필터링
WHERE s.name = '반도체' AND t.name = 'AI'
// ETF 정보와 관련 섹터, 기술 반환
RETURN e.name, e.code, s.name AS sector, t.name AS technology
"""
# 쿼리 실행
result = graph.query(cypher_query)

# 결과 출력
for row in result:
    print(row)

In [ ]:
# 특정 ETF와 유사한 ETF 추천 예시 (같은 섹터나 기술을 공유하는 ETF)
# 'TIGER AI반도체핵심공정' ETF와 공통 속성(섹터 또는 기술)을 공유하는 다른 ETF를 찾는 쿼리
cypher_query = """
// 기준 ETF와 다른 ETF 간의 공통 노드(섹터 또는 기술) 찾기
MATCH (e1:ETF {name: 'TIGER AI반도체핵심공정'})-[:INVESTS_IN|FOCUSES_ON]->(node)<-[:INVESTS_IN|FOCUSES_ON]-(e2:ETF)
// 자기 자신은 제외
WHERE e1 <> e2
// 유사 ETF 정보, 공통 속성 유형, 속성 이름 반환
RETURN e2.name AS similar_etf, e2.code AS code, 
       LABELS(node)[0] AS common_attribute, node.name AS attribute_name
// 5개 결과만 표시
LIMIT 5
"""
# Neo4j 데이터베이스에 쿼리 실행
result = graph.query(cypher_query)
# 결과 출력
for row in result:
    print(row)

#### 3) **전문(Full-Text) 검색 인덱스 설정**

- 전문 검색 인덱스는 텍스트를 단어(토큰)로 분리
- ETF 노드의 korean_name, english_name 속성에 대한 전문 검색 인덱스 생성

In [ ]:
# ETF 노드의 name과 english_name 속성에 대한 전문 검색 인덱스 생성
# CREATE FULLTEXT INDEX: 전문 검색(Full-Text Search)을 위한 인덱스를 생성하는 Cypher 명령어
# etf_name_fulltext: 생성할 인덱스의 이름으로, 나중에 이 이름으로 인덱스를 참조할 수 있음
# IF NOT EXISTS: 동일한 이름의 인덱스가 이미 존재하는 경우 오류 없이 건너뜀
# FOR (e:ETF): ETF 라벨을 가진 모든 노드에 대해 인덱스 적용
# ON EACH [e.name, e.english_name]: 각 ETF 노드의 korean_name과 english_name 속성을 인덱싱
etf_fulltext_index_query = """
// 전문 검색을 위한 인덱스 생성
// ETF 노드의 name과 english_name 속성에 대해 인덱싱
// 이미 존재하는 경우 오류 없이 건너뜀
CREATE FULLTEXT INDEX etf_name_fulltext IF NOT EXISTS 
FOR (e:ETF) ON EACH [e.name, e.english_name]
"""
graph.query(etf_fulltext_index_query)

In [ ]:
# 인덱스 생성 확인
graph.query("SHOW FULLTEXT INDEXES")

In [ ]:
# 전문 검색 테스트: AI 관련 ETF 검색
cypher_query = """
// 전문 검색 인덱스를 사용하여 ETF 노드 검색
CALL db.index.fulltext.queryNodes("etf_name_fulltext", $search_term)
// 검색된 노드와 관련도 점수 반환
YIELD node, score
// 결과 반환: ETF ID, 이름, 영문 이름, 검색 관련도 점수
RETURN node.id AS ETF_ID, node.name AS ETF_Name, 
       node.english_name AS ETF_English_Name, score AS SearchRelevance
// 검색 관련도 점수 기준으로 내림차순 정렬
ORDER BY SearchRelevance DESC
// 상위 5개 결과만 표시
LIMIT 5
"""
results = graph.query(cypher_query, params={"search_term": "ai"})

for result in results:
    print(f"{result['ETF_Name']} ({result['ETF_English_Name']}) - 관련도: {result['SearchRelevance']}")
print()

In [ ]:
# 전문 검색과 그래프 탐색 결합: AI 관련 ETF와 연결된 기술 찾기
cypher_query = """
// 전문 검색 인덱스를 사용하여 'AI'가 포함된 ETF 노드 검색
CALL db.index.fulltext.queryNodes("etf_name_fulltext", $search_term)
YIELD node as etf, score

// 검색된 ETF 노드에서 FOCUSES_ON 관계를 통해 연결된 Technology 노드 찾기
MATCH (etf)-[:FOCUSES_ON]->(tech:Technology)

// 결과 반환: ETF 이름, 검색 관련도 점수, 연결된 기술 목록
RETURN etf.name AS ETF_Name, score AS SearchRelevance, 
       collect(tech.name) AS RelatedTechnologies

// 검색 관련도 점수 기준으로 내림차순 정렬하고 상위 5개만 반환
ORDER BY SearchRelevance DESC
LIMIT 5
"""
results = graph.query(cypher_query, params={"search_term": "AI"})

for result in results:
    print(f"{result['ETF_Name']} (관련도: {result['SearchRelevance']})")
    if result['RelatedTechnologies']:
        print(f"  관련 기술: {', '.join(result['RelatedTechnologies'])}")
    else:
        print("  관련 기술 정보 없음")
    print()

In [ ]:
# 전문 검색 한글 테스트: '신재생 에너지' 관련 ETF 검색
cypher_query = """
// 전문 검색 인덱스를 사용하여 ETF 노드 검색
CALL db.index.fulltext.queryNodes("etf_name_fulltext", $search_term)
// 검색된 노드와 관련도 점수 반환
YIELD node, score
// 결과 반환: ETF ID, 이름, 영문 이름, 검색 관련도 점수
RETURN node.id AS ETF_ID, node.name AS ETF_Name, 
       node.english_name AS ETF_English_Name, score AS SearchRelevance
// 검색 관련도 점수 기준으로 내림차순 정렬
ORDER BY SearchRelevance DESC
// 상위 5개 결과만 표시
LIMIT 5
"""
results = graph.query(cypher_query, params={"search_term": "신재생 에너지"})

for result in results:
    print(f"{result['ETF_Name']} ({result['ETF_English_Name']}) - 관련도: {result['SearchRelevance']}")
    print()

In [ ]:
# 한국어를 지원하는 토크나이저를 적용하여 인덱스 생성 (기존 인덱스 삭제 후 생성)
# 먼저 기존 인덱스 삭제
cypher_query = """
DROP INDEX etf_name_fulltext IF EXISTS
"""
graph.query(cypher_query)

In [ ]:
# 한국어의 경우 cjk(Chinese, Japanese, Korean)가 통합된 analyzer를 지원
# 추가로 eventually_consistent 옵션을 사용하여 인덱스 성능 향상
cypher_query = """
CREATE FULLTEXT INDEX etf_name_fulltext IF NOT EXISTS
FOR (e:ETF) ON EACH [e.name, e.english_name]
OPTIONS {
  indexConfig: {
    `fulltext.analyzer`: 'cjk',
    `fulltext.eventually_consistent`: true
  }
}
"""
graph.query(cypher_query)

In [ ]:
# 인덱스 생성 확인
graph.query("SHOW FULLTEXT INDEXES")

In [ ]:
# 전문 검색 한글 테스트: '신재생 에너지' 관련 ETF 검색 (cjk analyzer로 한국어 지원)
cypher_query = """
// 전문 검색 인덱스를 사용하여 ETF 노드 검색
CALL db.index.fulltext.queryNodes("etf_name_fulltext", $search_term)
// 검색된 노드와 관련도 점수 반환
YIELD node, score
// 결과 반환: ETF ID, 이름, 영문 이름, 검색 관련도 점수
RETURN node.id AS ETF_ID, node.name AS ETF_Name, 
       node.english_name AS ETF_English_Name, score AS SearchRelevance
// 검색 관련도 점수 기준으로 내림차순 정렬
ORDER BY SearchRelevance DESC
// 상위 5개 결과만 표시
LIMIT 5
"""
results = graph.query(cypher_query, params={"search_term": "신재생 에너지"})

for result in results:
    print(f"{result['ETF_Name']} ({result['ETF_English_Name']}) - 관련도: {result['SearchRelevance']}")
print()

---

### 2.3 **Text2cypher 이용한 ETF 추천**

#### 1) **스키마 정보 확인**

- LLM이 Cypher 쿼리를 생성하려면 그래프 데이터베이스의 스키마 정보가 필요

In [ ]:
# 기본 스키마 정보 확인
graph.refresh_schema()
print(graph.schema)

#### 2) **GraphCypherQAChain 설정**

- `GraphCypherQAChain`은 LangChain에서 제공하는 체인으로, 자연어 질문을 Cypher 쿼리로 변환하고 그 결과를 바탕으로 답변을 생성

- 작동 과정:
    1. 사용자의 자연어 질문 입력
    2. LLM을 사용하여 질문을 Cypher 쿼리로 변환
    3. 생성된 Cypher 쿼리를 Neo4j 데이터베이스에 실행
    4. 쿼리 결과를 LLM에 전달하여 자연어 답변 생성

- 주요 구성 요소
    - `cypher_generation_chain`: 자연어를 Cypher 쿼리로 변환하는 체인
    - `qa_chain`: 쿼리 결과를 바탕으로 답변을 생성하는 체인
    - `graph`: Neo4j 그래프 데이터베이스 연결 객체
    - `graph_schema`: 그래프 데이터베이스의 스키마 정보

In [ ]:
from langchain_neo4j import GraphCypherQAChain
from langchain_google_genai import ChatGoogleGenerativeAI

# LLM 모델 설정
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0)

# GraphCypherQAChain 생성
cypher_chain = GraphCypherQAChain.from_llm(
    llm=llm,
    graph=graph,  
    validate_cypher=True,  # Cypher 쿼리 유효성 검사
    return_intermediate_steps=True,  # 중간 단계 결과 반환
    allow_dangerous_requests=True,  # DB에 영향을 줄 수 있음을 인지하고 쿼리 실행을 허용
    cypher_kwargs={"timeout": 60},  # Cypher 쿼리 실행 시간 제한
    top_k=5  # 반환할 최대 결과 수
)

In [ ]:
# 예시 질문 쿼리
cypher_chain.invoke({"query": "글로벌 인프라에 투자하는 ETF는 무엇인가요?"})

In [ ]:
from langchain.prompts import PromptTemplate
from langchain_neo4j import GraphCypherQAChain
from langchain_google_genai import ChatGoogleGenerativeAI

CYPHER_GENERATION_TEMPLATE = """
당신은 질문을 Cypher로 번역하는 Neo4j 전문가입니다.
스키마: {schema}

중요사항: 
- Cypher 쿼리에서 APOC 함수를 사용하지 마세요!
- 전문 검색을 위해서는 반드시 "etf_name_fulltext" 인덱스 이름만 사용하세요
- 전문 검색의 올바른 형식은 다음과 같습니다:
  CALL db.index.fulltext.queryNodes("etf_name_fulltext", "검색어")
  YIELD node, score
- 펀드 이름이나 ETF 이름에서 키워드를 검색할 때는 항상 전문 검색을 사용하세요
- "RETURN *" 또는 "RETURN DISTINCT *"를 변수 없이 사용하지 마세요
- 항상 "RETURN node.name, node.code"와 같이 명시적인 반환 변수를 지정하세요
- 모든 MATCH 패턴에는 최소한 하나의 변수가 정의되어 있어야 합니다
- RETURN 문에서 사용하기 전에 항상 변수를 정의하세요
- 변수 이름은 소문자로 사용하고 이름 지정에 일관성을 유지하세요
- 전문 검색 쿼리는 반드시 다음과 같은 형식으로 작성하세요:
  CALL db.index.fulltext.queryNodes("etf_name_fulltext", "검색어")
  YIELD node, score
  RETURN node.name AS ETF_Name, node.english_name AS ETF_English_Name, score AS SearchRelevance

질문: {question}
"""

cypher_prompt = PromptTemplate(
    template=CYPHER_GENERATION_TEMPLATE,
    input_variables=["schema", "question"],
)

# ETF QA를 위한 응답 생성 프롬프트
QA_TEMPLATE = """
당신은 ETF 데이터베이스 분석 전문가로서 ETF 데이터에 대한 명확하고 간결한 정보를 한국어로 제공합니다.

질문: {question}
검색 결과: {context}

응답 가이드라인:
- 검색 결과에서 핵심 정보를 요약하세요
- ETF 데이터에 대한 명확하고 객관적인 개요를 제공하세요
- 전문적이고 유익한 톤을 사용하세요
- ETF 데이터의 중요한 패턴이나 추세를 강조하세요
- 컨텍스트가 부족하거나 근거가 없을 경우 정보가 부족하다고 명확히 언급하세요
- 추측이나 개인적인 해석은 피하세요
- 데이터베이스에서 관련 정보를 찾을 수 없는 경우, 정확한 정보를 제공할 수 없다고 명시하세요

응답 형식:
- 간략한 발견 요약으로 시작하세요
- 여러 ETF가 발견된 경우 간결한 개요를 제공하세요
- 가독성을 위해 글머리 기호나 짧은 단락을 사용하세요
- 상장일, 수수료율, 투자 테마, 시장 등과 같은 관련 세부 정보를 포함하세요
- 수치 데이터나 기술 용어를 쉽게 이해할 수 있는 언어로 번역하세요

응답 구조 예시:
"분석 결과: [주요 발견 사항 요약]

주요 특징:
- [첫 번째 중요한 통찰]
- [두 번째 중요한 통찰]

추가 정보: [필요한 경우 추가 설명]"
"""

qa_prompt = PromptTemplate(
    template=QA_TEMPLATE,
    input_variables=["question", "context"],
)

# LLM 모델 설정
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0)

# GraphCypherQAChain 생성
cypher_chain = GraphCypherQAChain.from_llm(
    llm=llm,
    graph=graph,
    cypher_prompt=cypher_prompt,  # 커스텀 프롬프트 설정
    qa_prompt=qa_prompt,  # 커스텀 QA 프롬프트 설정
    validate_cypher=True,  # Cypher 쿼리 유효성 검사
    return_intermediate_steps=True,  # 중간 단계 결과 반환
    allow_dangerous_requests=True,  # DB에 영향을 줄 수 있음을 인지하고 쿼리 실행을 허용
    cypher_kwargs={"timeout": 60},  # Cypher 쿼리 실행 시간 제한
    top_k=5  # 반환할 최대 결과 수
)

In [ ]:
# 예시 질문 쿼리
cypher_chain.invoke({"query": "글로벌 인프라에 투자하는 ETF는 무엇인가요?"})

#### 3) **Few-shot Prompt**

In [ ]:
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_openai import OpenAIEmbeddings
from langchain_neo4j import Neo4jVector

# ETF 관련 few-shot 예제
ETF_EXAMPLES = [
    {
        "question": "인공지능 기술에 투자하는 ETF는 무엇인가요?",
        "query": "MATCH (e:ETF)-[:FOCUSES_ON]->(t:Technology {name: '인공지능'}) RETURN e.korean_name, e.code, e.basic_info LIMIT 5",
    },
    {
        "question": "미래에셋자산운용에서 관리하는 ETF 중 해외 시장에 투자하는 것은?",
        "query": "MATCH (e:ETF)-[:MANAGED_BY]->(:AssetManager {name: '미래에셋자산운용'}), (e)-[:INVESTS_IN]->(:Market {name: '해외'}) RETURN e.korean_name, e.code, e.base_market LIMIT 5",
    },
    {
        "question": "총 보수가 0.5% 미만인 주식형 ETF는 무엇인가요?",
        "query": "MATCH (e:ETF)-[:INVESTS_IN]->(:AssetClass {name: '주식'}) WHERE e.total_fee < 0.5 RETURN e.korean_name, e.code, e.total_fee ORDER BY e.total_fee ASC LIMIT 5",
    },
    {
        "question": "신재생에너지 테마 ETF 중 상장일이 가장 최근인 것은?",
        "query": "MATCH (e:ETF)-[:FOCUSES_ON]->(:InvestmentTheme {name: '신재생에너지'}) RETURN e.korean_name, e.code, e.listing_date ORDER BY e.listing_date DESC LIMIT 5",
    },
    {
        "question": "반도체 섹터에 투자하는 ETF 중 총 보수가 가장 낮은 것은?",
        "query": "MATCH (e:ETF)-[:BELONGS_TO]->(:Sector {name: '반도체'}) RETURN e.korean_name, e.code, e.total_fee ORDER BY e.total_fee ASC LIMIT 5",
    },
    {
        "question": "레버리지(추적배수 2배) ETF 중 국내 시장에 투자하는 것은?",
        "query": "MATCH (e:ETF)-[:INVESTS_IN]->(:Market {name: '국내'}) WHERE e.tracking_multiplier = 2.0 RETURN e.korean_name, e.code, e.tracking_multiplier LIMIT 5",
    },
    {
        "question": "원자력 테마에 투자하는 ETF 중 상장일이 가장 오래된 것은?",
        "query": "MATCH (e:ETF)-[:FOCUSES_ON]->(:InvestmentTheme {name: '원자력'}) RETURN e.korean_name, e.code, e.listing_date ORDER BY e.listing_date ASC LIMIT 5",
    },
    {
        "question": "비과세 혜택이 있는 ETF 중 해외 주식에 투자하는 것은?",
        "query": "MATCH (e:ETF)-[:INVESTS_IN]->(:AssetClass {name: '주식'}), (e)-[:INVESTS_IN]->(:Market {name: '해외'}) WHERE e.tax_type = '비과세' RETURN e.korean_name, e.code, e.tax_type LIMIT 5",
    },
    {
        "question": "교보악사자산운용에서 관리하는 ETF는 무엇인가요?",
        "query": "MATCH (e:ETF)-[:MANAGED_BY]->(:AssetManager {name: '교보악사자산운용'}) RETURN e.korean_name, e.code, e.fund_type LIMIT 5",
    },
    {
        "question": "통화 자산에 투자하는 ETF 중 총 보수가 가장 낮은 것은?",
        "query": "MATCH (e:ETF)-[:INVESTS_IN]->(:AssetClass {name: '통화'}) RETURN e.korean_name, e.code, e.total_fee ORDER BY e.total_fee ASC LIMIT 5",
    },
    {
        "question": "시스템 반도체와 AI 기술 모두에 투자하는 ETF는 무엇인가요?",
        "query": "MATCH (e:ETF)-[:BELONGS_TO]->(s:Sector {name: '시스템 반도체'}), (e)-[:FOCUSES_ON]->(t:Technology {name: 'AI'}) RETURN e.korean_name, e.code, s.name AS sector, t.name AS technology LIMIT 5",
    },
    {
        "question": "전문 검색으로 인공지능 관련 ETF를 찾고 관련도 점수를 알려주세요.",
        "query": "CALL db.index.fulltext.queryNodes('etf_name_fulltext', 'AI') YIELD node, score RETURN node.korean_name AS ETF_Name, node.english_name AS ETF_English_Name, score AS SearchRelevance ORDER BY SearchRelevance DESC LIMIT 5",
    },
    {
        "question": "AI 관련 ETF와 연결된 기술들을 모두 찾아주세요.",
        "query": "CALL db.index.fulltext.queryNodes('etf_name_fulltext', 'AI') YIELD node as etf, score MATCH (etf)-[:FOCUSES_ON]->(tech:Technology) RETURN etf.korean_name AS ETF_Name, score AS SearchRelevance, collect(tech.name) AS RelatedTechnologies ORDER BY SearchRelevance DESC LIMIT 5",
    },
    {
        "question": "인버스 ETF 중 거래량이 가장 많은 것은?",
        "query": "MATCH (e:ETF) WHERE e.tracking_multiplier < 0 RETURN e.korean_name, e.code, e.trading_volume ORDER BY e.trading_volume DESC LIMIT 5",
    },
    {
        "question": "배당수익률이 3% 이상인 ETF 중 국내 시장에 투자하는 것은?",
        "query": "MATCH (e:ETF)-[:INVESTS_IN]->(:Market {name: '국내'}) WHERE e.dividend_yield >= 3.0 RETURN e.korean_name, e.code, e.dividend_yield ORDER BY e.dividend_yield DESC LIMIT 5",
    }
] 


# 시맨틱 유사성 기반 예제 선택기 생성
example_selector = SemanticSimilarityExampleSelector.from_examples(
    ETF_EXAMPLES, 
    OpenAIEmbeddings(model="text-embedding-3-small"), 
    Neo4jVector, 
    k=3,  # 가장 유사한 3개의 예제 선택
    input_keys=["question"],
    url=os.environ["NEO4J_URI"],
    username=os.environ["NEO4J_USERNAME"],
    password=os.environ["NEO4J_PASSWORD"],
    database=os.environ["NEO4J_DATABASE"],  # Neo4j 데이터베이스 이름
)


# 예제 선택기 테스트
example_selector.select_examples({"question": "전문 검색으로 AI 관련 ETF를 찾아서 정리해주세요."})

In [ ]:
load_dotenv()

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Few-shot 예제 포맷팅 함수 정의
def format_few_shot_examples(examples):
    return "\n\n".join([
        f"Question: {ex['question']}\nCypher query: {ex['query']}"
        for ex in examples
    ])

# Cypher 쿼리 생성을 위한 프롬프트 템플릿 정의
CYPHER_TEMPLATE = """
당신은 질문을 Cypher로 번역하는 Neo4j 전문가입니다.
스키마: {schema}

중요사항: 
- Cypher 쿼리에서 APOC 함수를 사용하지 마세요!
- 전문 검색을 위해서는 반드시 "'etf_name_fulltext'" 인덱스 이름만 사용하세요
- 전문 검색의 올바른 형식은 다음과 같습니다:
  CALL db.index.fulltext.queryNodes("etf_name_fulltext", "검색어")
  YIELD node, score
- 펀드 이름이나 ETF 이름에서 키워드를 검색할 때는 항상 전문 검색을 사용하세요
- "RETURN *" 또는 "RETURN DISTINCT *"를 변수 없이 사용하지 마세요
- 항상 "RETURN node.name, node.code"와 같이 명시적인 반환 변수를 지정하세요
- 모든 MATCH 패턴에는 최소한 하나의 변수가 정의되어 있어야 합니다
- RETURN 문에서 사용하기 전에 항상 변수를 정의하세요
- 변수 이름은 소문자로 사용하고 이름 지정에 일관성을 유지하세요
- 전문 검색 쿼리는 반드시 다음과 같은 형식으로 작성하세요:
  CALL db.index.fulltext.queryNodes("etf_name_fulltext", "검색어")
  YIELD node, score
  RETURN node.name AS ETF_Name, node.english_name AS ETF_English_Name, score AS SearchRelevance

Few-shot Examples:
{few_shot_examples}

Question: {question}
Cypher query:
"""

CYPHER_GENERATION_PROMPT = PromptTemplate(
    input_variables=["schema", "question", "few_shot_examples"], 
    template=CYPHER_TEMPLATE
)

# Text2Cypher 체인 생성
text2cypher_chain = (
    {
        "question": lambda x: x["question"],
        "schema": lambda x: graph.schema,
        "few_shot_examples": lambda x: format_few_shot_examples(
            example_selector.select_examples({"question": x["question"]})
        )
    }
    | CYPHER_GENERATION_PROMPT 
    | llm 
    | StrOutputParser()
)

# Text2Cypher 체인 테스트
cypher_query = text2cypher_chain.invoke({"question": "전문 검색으로 AI 관련 ETF를 찾고 관련도 점수를 알려주세요."})
print(f"Cypher query: {cypher_query}")

# 쿼리 실행
result = graph.query(cypher_query)
print(result)

In [ ]:
from langchain_neo4j import GraphCypherQAChain
from langchain_google_genai import ChatGoogleGenerativeAI


# LLM 모델 설정
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0)

# 커스텀 프롬프트 적용 GraphCypherQAChain 생성
cypher_qa_chain = GraphCypherQAChain.from_llm(
    llm=llm,
    graph=graph,
    verbose=True,
    
    allow_dangerous_requests=True,
    return_intermediate_steps=True,
    
    # few_shot_examples 처리를 위한 cypher_llm_chain_kwargs 설정
    cypher_llm_chain_kwargs={
        "llm": llm,
        "prompt": CYPHER_GENERATION_PROMPT
    }
)

# 쿼리 실행 함수
def query_graph(question):
    # 쿼리 실행
    result = cypher_qa_chain.invoke({
        "query": question,
        "few_shot_examples": format_few_shot_examples(
            example_selector.select_examples({"question": question})
        )
    })
    return result

# 예시 쿼리 테스트
question = "미래에셋자산운용에서 운용하는 ETF는 무엇인가요??"
result = query_graph(question)


In [ ]:
# 결과 출력
for k, v in result.items():
    print(f"{k}:\n{v}")
    print("-" * 100)

In [ ]:
from langchain_neo4j import GraphCypherQAChain
from langchain_core.prompts import ChatPromptTemplate

# ETF 추천을 위한 프롬프트 템플릿 정의
ETF_RECOMMENDATION_TEMPLATE = """
당신은 ETF 데이터베이스 분석 전문가로서 ETF 데이터에 대한 명확하고 간결한 정보를 한국어로 제공합니다.

질문: {question}
검색 결과: {context}

응답 가이드라인:
- 검색 결과에서 핵심 정보를 요약하세요
- ETF 데이터에 대한 명확하고 객관적인 개요를 제공하세요
- 전문적이고 유익한 톤을 사용하세요
- ETF 데이터의 중요한 패턴이나 추세를 강조하세요
- 컨텍스트가 부족하거나 근거가 없을 경우 정보가 부족하다고 명확히 언급하세요
- 추측이나 개인적인 해석은 피하세요
- 데이터베이스에서 관련 정보를 찾을 수 없는 경우, 정확한 정보를 제공할 수 없다고 명시하세요

응답 형식:
- 간략한 발견 요약으로 시작하세요
- 여러 ETF가 발견된 경우 간결한 개요를 제공하세요
- 가독성을 위해 글머리 기호나 짧은 단락을 사용하세요
- 상장일, 수수료율, 투자 테마, 시장 등과 같은 관련 세부 정보를 포함하세요
- 수치 데이터나 기술 용어를 쉽게 이해할 수 있는 언어로 번역하세요

응답 구조 예시:
"분석 결과: [주요 발견 사항 요약]

주요 특징:
- [첫 번째 중요한 통찰]
- [두 번째 중요한 통찰]

추가 정보: [필요한 경우 추가 설명]"
"""

# ETF 추천 프롬프트 템플릿 생성
etf_recommendation_prompt = ChatPromptTemplate.from_template(ETF_RECOMMENDATION_TEMPLATE)


# ETF 추천을 위한 GraphCypherQAChain 생성
etf_qa_chain = GraphCypherQAChain.from_llm(
    llm=llm,
    graph=graph,  
    verbose=True,
    cypher_prompt=CYPHER_GENERATION_PROMPT,
    qa_prompt=etf_recommendation_prompt,
    validate_cypher=True,
    return_intermediate_steps=True,
    allow_dangerous_requests=True,
    top_k=5,
    return_direct=False,
    
    # few_shot_examples 처리를 위한 cypher_llm_chain_kwargs 설정
    cypher_llm_chain_kwargs={
        "llm": llm,
        "prompt": CYPHER_GENERATION_PROMPT
    }
)

# 쿼리 실행 함수
def query_graph(question):
    # 쿼리 실행
    result = etf_qa_chain.invoke({
        "query": question,
        "few_shot_examples": format_few_shot_examples(
            example_selector.select_examples({"question": question})
        )
    })
    return result

# 예시 쿼리 테스트
question = "전문 검색으로 AI 관련 ETF를 찾고 관련도 점수를 알려주세요."
result = query_graph(question)

In [ ]:
print(result['result'])

In [ ]:
print(result['intermediate_steps'])